In [1]:
import os
import sys
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm

w_dir = %pwd
work_dir = os.path.dirname(w_dir)
work_dir

'/work'

In [2]:
sys.path.append(w_dir+'/fgc_support_retri')
from fgc_support_retri.utils import *
from fgc_support_retri import config
from evaluation.eval import eval_sp_fgc, _update_sp
from evaluation.fgc_eval import *

In [3]:
data = json_load("prediction/ematch+sf_dev.json")
val_data = json_load(config.FGC_DEV)

In [4]:
STRING = '诺贝尔奖（瑞典语：Nobelpriset，挪威语：Nobelprisen，英语：NobelPrize），是根据瑞典化学家阿佛烈·诺贝尔的遗嘱于1901年开始表彰「在物理、化学、生理学或医学、文学和和平领域」作出最重要发现或发明的人，每年颁发五个奖项。\n阿佛烈·诺贝尔（瑞典语：Alfred Nobel，1833年10月21日－1896年12月10日）是一位瑞典化学家，同时也是发明者。诺贝尔终生未娶，亦无子嗣。在其逝世前，亲兄弟也早一步去世。由于诺贝尔终生主张和平主义，也因此他对于自己改良的炸药作为破坏及战争的用途始终感到痛心。在即将辞世之际，诺贝尔立下了遗嘱：「请将我的财产变做基金，每年用这个基金的利息作为奖金，奖励那些在前一年为人类做出卓越贡献的人。」\n1969年，瑞典中央银行设立了瑞典银行纪念诺贝尔经济科学奖，通称「诺贝尔经济学奖」。尽管与前述5种奖项「合称诺贝尔奖」，但实际上不是诺贝尔遗嘱中捐助的奖项。\n诺贝尔奖提名和评选程序\n9月：寄出邀请函。诺贝尔委员会向够资格提名的人发出邀请提名的信件。\n2月（次年）：提名截止。\n2月至3月：产生小名单。\n3月至8月：顾问审查小名单。\n10月：产生诺贝尔奖得主。\n12月：诺贝尔奖得主得到奖金。\n近50年以来，诺贝尔基金会明文规定，不论是公共场合还是私人场合，提名过程和人选都要严格保密，包括提名者人选和被提名者人选。很多人在各种场合声称自己曾因某成果而获得诺贝尔奖的提名但未获奖，应属谣言。'

In [5]:
STRING[123:131]

'项。\n阿佛烈·诺'

In [6]:
# get_SHINT(data)

In [7]:
get_answer_sp(data, force=True)

100%|██████████| 247/247 [00:00<00:00, 16374.16it/s]

D097Q08
{'text': '\n3月至8月：顾问审查小名单。', 'start': 481, 'end': 496, 'IE': {'NER': [{'id': 'D0-S0-M0', 'string': '3月', 'type': 'DATE', 'char_b': 1, 'char_e': 3}, {'id': 'D0-S0-M1', 'string': '至', 'type': 'MISC', 'char_b': 3, 'char_e': 4}, {'id': 'D0-S0-M2', 'string': '8月', 'type': 'DATE', 'char_b': 4, 'char_e': 6}, {'id': 'D0-S0-M3', 'string': '顾问', 'type': 'TITLE', 'char_b': 7, 'char_e': 9}], 'COREF': {}, 'RELATION': [], 'TOKEN': [{'word': '3月', 'char_b': 1, 'char_e': 3, 'pos': 'NT'}, {'word': '至', 'char_b': 3, 'char_e': 4, 'pos': 'CC'}, {'word': '8月', 'char_b': 4, 'char_e': 6, 'pos': 'NT'}, {'word': '：', 'char_b': 6, 'char_e': 7, 'pos': 'PU'}, {'word': '顾问', 'char_b': 7, 'char_e': 9, 'pos': 'NN'}, {'word': '审查', 'char_b': 9, 'char_e': 11, 'pos': 'VV'}, {'word': '小', 'char_b': 11, 'char_e': 12, 'pos': 'JJ'}, {'word': '名单', 'char_b': 12, 'char_e': 14, 'pos': 'NN'}, {'word': '。', 'char_b': 14, 'char_e': 15, 'pos': 'PU'}]}}
{'text': '10月', 'text_cn': '10月', 'start': 495, 'end': 498}
D097Q09
{'

In [16]:
object_data = []
yesno_data = []
kinship_data = []
others_data = []
for d in data:
    atype = d['QUESTIONS'][0]['ATYPE']
    if atype == 'Object':
        object_data.append(d)
    elif atype == 'YesNo':
        yesno_data.append(d)
    elif atype == 'Kinship':
        kinship_data.append(d)
    else:
        others_data.append(d)

In [17]:
print(len(object_data))
print(len(yesno_data))
print(len(kinship_data))
print(len(others_data))

49
28
32
138


In [27]:
all_items, all_sp_predictions, all_answer_sp = eval_from_threshold(others_data, threshold=0.5)

100%|██████████| 138/138 [00:00<00:00, 42310.96it/s]


In [28]:
metrics = eval_sp_fgc(all_items, all_sp_predictions)

{'sp_em': 0.162, 'sp_prec': 0.621, 'sp_recall': 0.7, 'sp_f1': 0.611}


In [29]:
metrics = eval_sp_fgc(all_answer_sp, all_sp_predictions)

{'sp_em': 0.14, 'sp_prec': 0.402, 'sp_recall': 0.771, 'sp_f1': 0.483}


In [11]:
len(all_answer_sp)

239

In [12]:
def print_sents_with_qid(qid, data):
    for d in data:
        for q in d['QUESTIONS']:
            if q['QID'] == qid:
                for s_i, s in enumerate(d['SENTS']):
#                     print((s_i, s['text'], q['sp_scores'][s_i]))
                    print("{}:{}".format(s_i, s['text'].strip()))

In [13]:
def count_string_in_d(text, qid, data):
    for d in data:
        for q in d['QUESTIONS']:
            count = 0
            if q['QID'] == qid:
                for s_i, sent in enumerate(d['SENTS']):
                    if text in sent['text']:
                        print((s_i, sent['text']))
                        count += 1
                return count, len(d['SENTS'])

In [14]:
count_string_in_d('东坡', 'D001Q08', val_data)

(4, '号东坡居士、铁冠道人。')
(10, '有《东坡先生大全集》及《东坡乐府》词集传世，')


(2, 36)

In [24]:
print_sents_with_qid('D037Q19', data)

0:称直系血亲者，
1:谓己身所从出或从己身所出之血亲。
2:包括父亲、母亲、儿子、女儿、祖父、祖母、外祖父、外祖母、孙子、孙女、外孙、外孙女。
3:称旁系血亲者，
4:谓非直系血亲，
5:而与己身出于同源之血亲。
6:包括同源于父母的兄弟姊妹（含同父异母、同母异父的兄弟姊妹）以及不同辈的叔父、伯父、舅父、姑母、姨母、侄子、侄女、外甥、外甥女等等。
7:旁系血亲中的同辈如兄弟姊妹 - 也作兄弟姐妹，
8:指有相同父亲和母亲的人。
9:包括双胞胎兄弟姐妹、同父异母兄弟姐妹以及同母异父兄弟姐妹。
10:堂表之分，
11:同宗是堂，异宗是表。
12:即父系的男丁是堂，
13:其他是表。要注意是表姑是父系的表姐妹，
14:而不是父系的姐妹。
15:堂兄弟姊妹 - 父系血亲的长辈（伯、姆、叔、婶）的儿子或女儿，
16:口语时一些地方不带「堂」字。
17:表兄弟姊妹 - 父系血亲的长辈（姑）的儿子或女儿和母系血亲的长辈（舅、妗、姨）的儿子或女儿
旁系血亲的晚辈有：
姪，
18:或作侄，是一种亲属关系的称谓，
19:指哥哥或弟弟的子女。
20:男性的姪称为姪子或姪儿，
21:女性的姪则称为姪女；
22:自己为男性，
23:对方称呼自己作伯父或叔父，
24:自己为女性，
25:对方称呼自己作姑母。
26:现代除非从母姓,否则侄是一定与本人同姓,类同堂的性质。
27:甥是一个亲属称谓，
28:指姊姊或妹妹的子女。
29:男性（舅父）称姊妹的子女为外甥／外甥女，
30:女性（姨母）称姊妹的子女为姨甥／姨甥女。


In [16]:
def print_analysis_from_qid(qid, data):
    for d in data:
        for q in d['QUESTIONS']:
            if q['QID'] == qid:
                print(q['QID'])
                print(q['QTEXT_CN'])
                print("atype:{}".format(q['ATYPE']))
                print("SHINT:{}".format(q['SHINT']))
                print("answer_sp:{}".format(q['answer_sp']))
                print("sp:{}".format(q['sp']))
                print("answer:{}".format(q['ANSWER']))
                all_set = set(q['SHINT']) | set(q['sp'])
                for sp_i in range(min(all_set), max(all_set) + 1):
                    print(q['sp_scores'][sp_i])
                    print_sent_ie(sp_i, d)
                    print()

In [17]:
print_analysis_from_qid('D073Q11', data)

D073Q11
菲律宾的鲸鲨保育政策为何?
atype:Misc
SHINT:[24]
answer_sp:[24]
sp:[0, 16, 22, 23, 24]
answer:[{'ATEXT': '禁止所有有關鯨鯊的商業獵捕、販賣、輸入與輸出活動。', 'ATEXT_CN': '禁止所有有关鲸鲨的商业猎捕、贩卖、输入与输出活动。', 'ATOKEN': [{'text': '禁止所有有關鯨鯊的商業獵捕', 'text_cn': '禁止所有有关鲸鲨的商业猎捕', 'start': -1, 'end': 12}, {'text': '販賣', 'text_cn': '贩卖', 'start': 553, 'end': 555}, {'text': '輸入', 'text_cn': '输入', 'start': 556, 'end': 558}, {'text': '輸出活動。', 'text_cn': '输出活动。', 'start': -1, 'end': 4}]}]
0.8283123970031738
sentence0
鲸鲨（学名：Rhincodon typus、英语：whale shark），
{'id': 'D0-S0-M0', 'string': '英语', 'type': 'DEMONYM', 'char_b': 22, 'char_e': 24}

0.2013264149427414
sentence1
又名鲸鲛、豆腐鲨、大憨鲨，

0.028428321704268456
sentence2
是目前世界上体型最大的鱼类，
{'id': 'D0-S0-M0', 'string': '目前', 'type': 'DATE', 'char_b': 1, 'char_e': 3}

0.07146327942609787
sentence3
属于软骨鱼纲须鲨目鲸鲨科。

0.0033302626106888056
sentence4
截至目前为止，
{'id': 'D0-S0-M0', 'string': '目前', 'type': 'DATE', 'char_b': 2, 'char_e': 4}

0.0011129953199997544
sentence5
生物学家实际记录到的最大尺寸为12.65公尺长，
{'id': 'D0-S0-M0', 

In [18]:
data[100]

{'DID': 'D091',
 'QUESTIONS': [{'QID': 'D091Q11',
   'QTYPE': '基礎題',
   'ATYPE': 'Kinship',
   'AMODE': ['Kinship', 'Multi-Spans-Extraction'],
   'QTEXT': '先生的奶奶要怎麼稱呼她?',
   'QTEXT_CN': '先生的奶奶要怎么称呼她?',
   'SENTS': [{'text': '先生的奶奶要怎么称呼她?',
     'start': 0,
     'end': 12,
     'IE': {'NER': [],
      'COREF': {},
      'RELATION': [],
      'TOKEN': [{'word': '先生', 'char_b': 0, 'char_e': 2, 'pos': 'NN'},
       {'word': '的', 'char_b': 2, 'char_e': 3, 'pos': 'DEG'},
       {'word': '奶奶', 'char_b': 3, 'char_e': 5, 'pos': 'NN'},
       {'word': '要', 'char_b': 5, 'char_e': 6, 'pos': 'VV'},
       {'word': '怎么', 'char_b': 6, 'char_e': 8, 'pos': 'AD'},
       {'word': '称呼', 'char_b': 8, 'char_e': 10, 'pos': 'VV'},
       {'word': '她', 'char_b': 10, 'char_e': 11, 'pos': 'PN'},
       {'word': '?', 'char_b': 11, 'char_e': 12, 'pos': 'PU'}]}}],
   'SHINT': [2, 3],
   'ANSWER': [{'ATEXT': '母親、媽媽、娘及婆婆',
     'ATEXT_CN': '母亲、妈妈、娘及婆婆',
     'ATOKEN': [{'text': '母親', 'text_cn': '母亲', 'start': -1, 'e

In [19]:
d = data[100]
for q in d['QUESTIONS']:
    print(q['QID'])
    print(q['QTEXT_CN'])
    print("atype:{}".format(q['ATYPE']))
    print("SHINT:{}".format(q['SHINT']))
    print("answer_sp:{}".format(q['answer_sp']))
    print("sp:{}".format(q['sp']))
    print()
    all_set = set(q['SHINT']) | set(q['sp'])
    for sp_i in range(min(all_set), max(all_set) + 1):
        print(q['sp_scores'][sp_i])
        print(get_sent_ie(sp_i, d))

D091Q11
先生的奶奶要怎么称呼她?
atype:Kinship
SHINT:[2, 3]
answer_sp:[2, 3]
sp:[3, 12, 38]

0.16844238340854645
sentence2
当对象是丈夫的祖母，

0.7532238960266113
sentence3
我们要称呼她为祖姑(奶奶)。

0.02204076759517193
sentence4
而我们自己要称作是孙媳，

0.0004974870244041085
sentence5
对第三者介绍他们时，
{'id': 'D0-S0-M0', 'string': '第三', 'type': 'ORDINAL', 'char_b': 1, 'char_e': 3}

0.028722232207655907
sentence6
会说家祖翁家祖姑。
{'id': 'D0-S0-M0', 'string': '家祖翁', 'type': 'PER', 'char_b': 2, 'char_e': 5}

0.0007687144679948688
sentence7
当我们称呼别人的家祖翁家祖姑时，

0.0033920714631676674
sentence8
会说令祖翁令祖姑。

0.0011073179775848985
sentence9

当对象是丈夫的父亲，

0.0012314891209825873
sentence10
我们要称呼他为父亲(爸爸/爹/公公/老爷)；

0.1631346493959427
sentence11
当对象是丈夫的母亲，

0.6835855841636658
sentence12
我们要称呼她为母亲(妈妈/娘/婆婆/奶奶(广东人用))。
{'id': 'D0-S0-M0', 'string': '广东人', 'type': 'DEMONYM', 'char_b': 21, 'char_e': 24}

0.003852853551506996
sentence13
而我们自己要称作是媳，

0.0004974870244041085
sentence14
对第三者介绍他们时，
{'id': 'D0-S0-M0', 'string': '第三', 'type': 'ORDINAL', 'char_b': 1, 'char_e':

In [20]:
def print_analysis_from_qid(qid, data):
    for d in data:
        for q in d['QUESTIONS']:
            if q['QID'] == qid:
                print(q['QID'])
                print(q['QTEXT_CN'])
                print("sent_len:{}".format(len(d['SENTS'])))
                print("atype:{}".format(q['ATYPE']))
                print("SHINT:{}".format(q['SHINT']))
                print("answer_sp:{}".format(q['answer_sp']))
                print("sp:{}".format(q['sp']))
                print()
                all_set = set(q['SHINT']) | set(q['sp'])
                for sp_i in range(min(all_set), max(all_set) + 1):
                    print(q['sp_scores'][sp_i])
                    print(get_sent_ie(sp_i, d))

In [21]:
print_analysis_from_qid('D031Q01', data)

D031Q01
新版的北美贸易协定英文简称为何?
sent_len:15
atype:Object
SHINT:[7, 14]
answer_sp:[14]
sp:[11]

0.07138276100158691
sentence7
成为美、加、墨3国去年谈判以期达成一项新的北美贸易协定的重大障碍。
{'id': 'D0-S0-M0', 'string': '美', 'type': 'GPE', 'char_b': 2, 'char_e': 3}
{'id': 'D0-S0-M1', 'string': '加', 'type': 'GPE', 'char_b': 4, 'char_e': 5}
{'id': 'D0-S0-M2', 'string': '墨', 'type': 'GPE', 'char_b': 6, 'char_e': 7}
{'id': 'D0-S0-M3', 'string': '3', 'type': 'NUMBER', 'char_b': 7, 'char_e': 8}
{'id': 'D0-S0-M4', 'string': '去年', 'type': 'DATE', 'char_b': 9, 'char_e': 11}
{'id': 'D0-S0-M5', 'string': '一', 'type': 'NUMBER', 'char_b': 17, 'char_e': 18}
{'id': 'D0-S0-M6', 'string': '北美', 'type': 'LOC', 'char_b': 21, 'char_e': 23}

0.0025384051259607077
sentence8

加拿大总理杜鲁道（Justin Trudeau）在渥太华告诉记者，
{'id': 'D0-S0-M0', 'string': '加拿大', 'type': 'LOC', 'char_b': 1, 'char_e': 4}
{'id': 'D0-S0-M1', 'string': '总理', 'type': 'TITLE', 'char_b': 4, 'char_e': 6}
{'id': 'D0-S0-M2', 'string': '杜鲁道', 'type': 'PER', 'char_b': 6, 'char_e': 9}
{'id': 'D

In [22]:
len(data)

247

In [23]:
f = open('prediction/ematch+sf.analysis', 'w')
f.write(get_analysis(data))
f.close()

# Excel

In [10]:
qid = []
qtext = []
shint_scores = []
sp = []
answer_sp = []
for d in data:
    for q in d['QUESTIONS']:
        qid.append(q['QID'])
        qtext.append(q['QTEXT_CN'])
        shint_score = [(shint_i, d['SENTS'][shint_i]['text'], q['sp_scores'][shint_i]) for shint_i in q['SHINT']]
        shint_scores.append(shint_score)
        sp_scores = [(sp_i, d['SENTS'][sp_i]['text'], q['sp_scores'][sp_i]) for sp_i in q['sp']]
        sp.append(sp_scores)

In [11]:
sp[0]

[(0, '苏轼（1037年1月8日－1101年8月24日），', 0.7372738122940063)]

In [12]:
df = pd.DataFrame({'qid': qid, 'error_type':[0]*len(qid)})

In [13]:
df.head(5)

,qid,error_type
0,D001Q01,0
1,D001Q02,0
2,D001Q03,0
3,D001Q04,0
4,D001Q05,0


In [32]:
df.to_excel('entity_old.xlsx')

In [88]:
df = pd.read_excel('entity_old.xlsx')

In [90]:
df.error_type

0      4
1      4
2      1
3      4
4      3
      ..
242    0
243    0
244    0
245    0
246    0
Name: error_type, Length: 247, dtype: int64

In [91]:
error_count = [0]*7
for type_i in df.error_type:
    error_count[type_i] += 1

In [92]:
error_count

[208, 2, 4, 9, 13, 8, 3]